In [1]:
import os
from pathlib import Path
import re

from dotenv import find_dotenv, load_dotenv
from rich import print as rprint
from rich.pretty import Pretty, pprint
from rich.text import Text

In [2]:
load_dotenv(find_dotenv('.env'))

True

In [3]:
os.environ["LANGCHAIN_PROJECT"] = "RAG From Scratch: Part 3 (Retrieval)"

In [4]:
DATA_PATH = Path('./data')
VECTORSTORE_PATH = DATA_PATH / 'vectorstore'

# Part 3: Retrieval

![](images/03-01-retrieval.png)

![](images/03-02-retrieval.png)

## Configure components

In [5]:
from langchain_openai import OpenAIEmbeddings

In [6]:
embeddings_model_name = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embeddings_model_name)
len(embeddings.embed_query("Hello"))

1536

## Load documents

In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
len(docs)

1

In [9]:
print(docs[0].page_content[:1000])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

## Split documents

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
len(splits)

66

## Store documents

**Similarity search**

In [12]:
from langchain_core.vectorstores import InMemoryVectorStore

In [13]:
vectorstore = InMemoryVectorStore(embeddings)
doc_ids = vectorstore.add_documents(documents=splits)
len(doc_ids), len(vectorstore.store)

(66, 66)

In [14]:
records = list(vectorstore.store.items())
Pretty(records[0], max_length=20, no_wrap=False)

(
    '06db5411-f902-41db-befe-567ebed11703',
    {
        'id': '06db5411-f902-41db-befe-567ebed11703',
        'vector': [
            0.009637040086090565,
            0.020359551534056664,
            0.0416097454726696,
            -0.004188807215541601,
            0.003524303901940584,
            0.0019100125646218657,
            -0.014472954906523228,
            0.03676687180995941,
            -0.01905141957104206,
            0.0588381290435791,
            0.037240028381347656,
            -0.052575793117284775,
            -0.02922423742711544,
            -0.029363399371504784,
            -0.04369719326496124,
            -0.049096718430519104,
            -0.01288649719208479,
            -0.024103038012981415,
            0.042333394289016724,
            0.016226409003138542,
            ... +1516
        ],
        'text': 'LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  
Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. 
Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The 
potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed 
as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM 
functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: 
The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex 
tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn 
from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory',
        'metadata': {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
    }
)

## Semantic search (vector store)

In [15]:
query = "What is Task Decomposition?"
relevant_documents = vectorstore.similarity_search(query)
len(relevant_documents)

4

In [16]:
for document in relevant_documents:
    rprint(document)

Document(
    id='7cd63e32-c110-4e3b-a055-2acfcfc8a659',
    metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
    page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at 
each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, 
creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with 
each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM 
with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using 
task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'
)

Document(
    id='6fbc5dca-2f04-4605-93e7-adfb52fc0edc',
    metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
    page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA 
complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask 
Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing 
model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time 
computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple 
manageable tasks and shed lights into an interpretation of the model’s thinking process.'
)

Document(
    id='27935b69-f72e-4cef-a560-59743b298ada',
    metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
    page_content='The AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": 
dependency_task_ids, "args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes 
the id of the previous task which generates a new resource that the current task relies on. A special tag 
"-task_id" refers to the generated text image, audio and video in the dependency task with id as task_id. The task 
MUST be selected from the following options: {{ Available Task List }}. There is a logical relationship between 
tasks, please note their order. If the user input can\'t be parsed, you need to reply empty JSON. Here are several 
cases for your reference: {{ Demonstrations }}. The chat history is recorded as {{ Chat History }}. From this chat 
history, you can find the path of the user-mentioned resources for your task planning.'
)

Document(
    id='7660a1be-2600-4440-935c-723bd117bb13',
    metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
    page_content='Fig. 11. Illustration of how HuggingGPT works. (Image source: Shen et al. 2023)\nThe system 
comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks.
There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot 
examples to guide LLM to do task parsing and planning.\nInstruction:'
)

In [17]:
rprint(query)

chunk_pattern = re.compile(r'^Chunk \d+.*:$', flags=re.MULTILINE)
terms_pattern = re.compile(rf'\b({"|".join(query.split())})\b', flags=re.IGNORECASE)

for chunk_id, document in enumerate(relevant_documents, start=1):
    text = Text(f"Chunk {chunk_id}:\n{document.page_content}")
    text.highlight_regex(chunk_pattern, "bold green")
    text.highlight_regex(terms_pattern, "underline red")
    rprint(text)

What is Task Decomposition?

Chunk 1:
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first
decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree 
structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state 
evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals 
for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or
(3) with human inputs.

Chunk 2:
Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance 
on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to 
decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and 
shed lights into an interpretation of the model’s thinking process.

Chunk 3:
The AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": dependency_task_ids, 
"args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes the id of the previous 
task which generates a new resource that the current task relies on. A special tag "-task_id" refers to the 
generated text image, audio and video in the dependency task with id as task_id. The task MUST be selected from the
following options: {{ Available Task List }}. There is a logical relationship between tasks, please note their 
order. If the user input can't be parsed, you need to reply empty JSON. Here are several cases for your reference: 
{{ Demonstrations }}. The chat history is recorded as {{ Chat History }}. From this chat history, you can find the 
path of the user-mentioned resources for your task planning.

Chunk 4:
Fig. 11. Illustration of how HuggingGPT works. (Image source: Shen et al. 2023)
The system comprises of 4 stages:
(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four 
attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to 
guide LLM to do task parsing and planning.
Instruction:

## Semantic search (retriever)

In [18]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)
retriever

VectorStoreRetriever(tags=['InMemoryVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_core.vectorstores.in_memory.InMemoryVectorStore object at 0x7fca9ceeb090>, search_kwargs={'k': 4})

In [19]:
query = "What is Task Decomposition?"
relevant_documents = retriever.invoke(query)
len(relevant_documents)

4

In [20]:
for document in relevant_documents:
    rprint(document)

Document(
    id='7cd63e32-c110-4e3b-a055-2acfcfc8a659',
    metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
    page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at 
each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, 
creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with 
each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM 
with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using 
task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'
)

Document(
    id='6fbc5dca-2f04-4605-93e7-adfb52fc0edc',
    metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
    page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA 
complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask 
Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing 
model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time 
computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple 
manageable tasks and shed lights into an interpretation of the model’s thinking process.'
)

Document(
    id='27935b69-f72e-4cef-a560-59743b298ada',
    metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
    page_content='The AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": 
dependency_task_ids, "args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes 
the id of the previous task which generates a new resource that the current task relies on. A special tag 
"-task_id" refers to the generated text image, audio and video in the dependency task with id as task_id. The task 
MUST be selected from the following options: {{ Available Task List }}. There is a logical relationship between 
tasks, please note their order. If the user input can\'t be parsed, you need to reply empty JSON. Here are several 
cases for your reference: {{ Demonstrations }}. The chat history is recorded as {{ Chat History }}. From this chat 
history, you can find the path of the user-mentioned resources for your task planning.'
)

Document(
    id='7660a1be-2600-4440-935c-723bd117bb13',
    metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'},
    page_content='Fig. 11. Illustration of how HuggingGPT works. (Image source: Shen et al. 2023)\nThe system 
comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks.
There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot 
examples to guide LLM to do task parsing and planning.\nInstruction:'
)

In [21]:
rprint(query)

chunk_pattern = re.compile(r'^Chunk \d+.*:$', flags=re.MULTILINE)
terms_pattern = re.compile(rf'\b({"|".join(query.split())})\b', flags=re.IGNORECASE)

for chunk_id, document in enumerate(relevant_documents, start=1):
    text = Text(f"Chunk {chunk_id}:\n{document.page_content}")
    text.highlight_regex(chunk_pattern, "bold green")
    text.highlight_regex(terms_pattern, "underline red")
    rprint(text)

What is Task Decomposition?

Chunk 1:
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first
decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree 
structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state 
evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals 
for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or
(3) with human inputs.

Chunk 2:
Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance 
on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to 
decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and 
shed lights into an interpretation of the model’s thinking process.

Chunk 3:
The AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": dependency_task_ids, 
"args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes the id of the previous 
task which generates a new resource that the current task relies on. A special tag "-task_id" refers to the 
generated text image, audio and video in the dependency task with id as task_id. The task MUST be selected from the
following options: {{ Available Task List }}. There is a logical relationship between tasks, please note their 
order. If the user input can't be parsed, you need to reply empty JSON. Here are several cases for your reference: 
{{ Demonstrations }}. The chat history is recorded as {{ Chat History }}. From this chat history, you can find the 
path of the user-mentioned resources for your task planning.

Chunk 4:
Fig. 11. Illustration of how HuggingGPT works. (Image source: Shen et al. 2023)
The system comprises of 4 stages:
(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four 
attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to 
guide LLM to do task parsing and planning.
Instruction: